In [1]:
import polars as pl
from pathlib import Path
from polimi.utils._urm import train_recommender
from RecSys_Course_AT_PoliMi.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from RecSys_Course_AT_PoliMi.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from RecSys_Course_AT_PoliMi.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from RecSys_Course_AT_PoliMi.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from RecSys_Course_AT_PoliMi.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_AsySVD_Cython, MatrixFactorization_BPR_Cython
from RecSys_Course_AT_PoliMi.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender
from RecSys_Course_AT_PoliMi.Recommenders.Neural.MultVAERecommender import MultVAERecommender
from RecSys_Course_AT_PoliMi.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, MultiThreadSLIM_SLIMElasticNetRecommender
from RecSys_Course_AT_PoliMi.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from RecSys_Course_AT_PoliMi.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

In [13]:
x = pl.read_parquet(Path('/Users/lorecampa/Desktop/Projects/RecSysChallenge2024/experiments') / 'preprocessing_moving_window_2024-05-23_11-27-50'/'fold_1'/'train_ds.parquet')
x.head(1)

impression_id,article,user_id,impression_time,device_type,read_time,scroll_percentage,is_sso_user,gender,age,is_subscriber,target,trendiness_score_3d,weekday,hour,premium,category,sentiment_score,sentiment_label,num_images,title_len,subtitle_len,body_len,num_topics,article_delay_days,article_delay_hours,Entity_ORG_Present,Entity_PROD_Present,Entity_PER_Present,Entity_EVENT_Present,Entity_MISC_Present,Entity_LOC_Present,last_session_duration,mean_prev_sessions_duration,last_session_time_hour_diff,is_new_article,is_already_seen_article,…,total_pageviews/inviews_minus_median_impression,mean_JS_minus_median_impression,mean_topic_model_cosine_minus_median_impression,topics_cosine_minus_median_impression,article_delay_hours_minus_median_impression,total_pageviews_minus_median_impression,total_inviews_minus_median_impression,trendiness_score_category_minus_median_impression,std_JS_minus_median_impression,total_read_time_minus_median_impression,endorsement_10h_leak_minus_median_impression,trendiness_score_3d_leak_minus_median_impression,clicked_count_minus_median_impression,inview_count_minus_median_impression,category_diversity_impression,sentiment_label_diversity_impression,article_type_diversity_impression,endorsement_20h_articleuser_l_inf_articleuser,sum_RP3betaRecommender_ner_scores_l_inf_impression,max_RP3betaRecommender_ner_scores_l_inf_impression,mean_RP3betaRecommender_ner_scores_l_inf_impression,contrastive_vector_scores_weighted_scroll_percentage_fixed_mmnorm_l1_w_mean_l_inf_impression,xlm_roberta_base_scores_weighted_scroll_percentage_fixed_mmnorm_l1_w_mean_l_inf_impression,image_embeddings_scores_weighted_scroll_percentage_fixed_mmnorm_l1_w_mean_l_inf_impression,bert_base_multilingual_cased_scores_weighted_scroll_percentage_fixed_mmnorm_l1_w_mean_l_inf_impression,contrastive_vector_scores_weighted_scroll_percentage_fixed_mmnorm_l1_w_max_l_inf_impression,xlm_roberta_base_scores_weighted_scroll_percentage_fixed_mmnorm_l1_w_max_l_inf_impression,image_embeddings_scores_weighted_scroll_percentage_fixed_mmnorm_l1_w_max_l_inf_impression,bert_base_multilingual_cased_scores_weighted_scroll_percentage_fixed_mmnorm_l1_w_max_l_inf_impression,contrastive_vector_scores_weighted_scroll_percentage_fixed_mmnorm_l1_w_std_l_inf_impression,xlm_roberta_base_scores_weighted_scroll_percentage_fixed_mmnorm_l1_w_std_l_inf_impression,image_embeddings_scores_weighted_scroll_percentage_fixed_mmnorm_l1_w_std_l_inf_impression,bert_base_multilingual_cased_scores_weighted_scroll_percentage_fixed_mmnorm_l1_w_std_l_inf_impression,contrastive_vector_scores_weighted_scroll_percentage_fixed_mmnorm_l1_w_median_l_inf_impression,xlm_roberta_base_scores_weighted_scroll_percentage_fixed_mmnorm_l1_w_median_l_inf_impression,image_embeddings_scores_weighted_scroll_percentage_fixed_mmnorm_l1_w_median_l_inf_impression,bert_base_multilingual_cased_scores_weighted_scroll_percentage_fixed_mmnorm_l1_w_median_l_inf_impression
u32,i32,u32,datetime[μs],i8,f32,f32,bool,i8,i8,bool,i8,i16,i8,i8,bool,i16,f32,str,u32,u8,u8,u16,u32,i16,i32,bool,bool,bool,bool,bool,bool,i8,f32,i16,bool,bool,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,u8,u8,u8,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
470303568,9771042,2456550,2023-05-18 07:00:14,2,3.0,null,false,2,null,false,0,96,4,7,false,118,0.6719,"""Positive""",1,4,17,200,3,0,0,true,false,true,true,false,false,0,0.0,14,false,false,…,-0.013114,-0.033377,0.012717,0.124914,-1.0,-23633.5,-41238.5,15.0,-0.037947,-1.748765e6,53.0,3.5,-66.0,29.5,6,3,1,1.0,0.0,0.0,0.0,1.0,0.99994,0.901687,0.999985,0.758922,0.999893,0.949899,1.0,1.0,0.999954,0.958381,1.0,1.0,0.999906,0.575378,0.997759


In [15]:
a = x.select('impression_id', 'article', 'user_id', *[col for col in x.columns if '_ner_scores' in col])
a

impression_id,article,user_id,sum_RP3betaRecommender_ner_scores_l_inf_impression,max_RP3betaRecommender_ner_scores_l_inf_impression,mean_RP3betaRecommender_ner_scores_l_inf_impression
u32,i32,u32,f32,f32,f32
470303568,9771042,2456550,0.0,0.0,0.0
470303568,9769306,2456550,0.0,0.0,0.0
470303569,9769553,2456550,0.0,0.0,0.0
157803571,9770989,2226651,0.0,0.0,0.0
470303569,9770798,2456550,0.0,0.0,0.0
…,…,…,…,…,…
567361947,9775402,667805,0.164644,0.292057,0.146351
567361947,9754160,667805,0.758724,0.810411,0.551799
567361947,9775430,667805,0.0,0.0,0.0


In [5]:
dpath = Path('/mnt/ebs_volume/recsys2024/dataset')

dtype = 'small'
articles = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/articles.parquet')

behaviors_train = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/train/behaviors.parquet')
history_train = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/train/history.parquet')

behaviors_val = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/validation/behaviors.parquet')
history_val = pl.read_parquet(f'{dpath}/ebnerd_{dtype}/validation/history.parquet')

FileNotFoundError: No such file or directory (os error 2): /mnt/ebs_volume/recsys2024/dataset/ebnerd_small/articles.parquet

In [5]:
import os
os.listdir(None)

['test_feature_urm_scores.ipynb',
 'urm_ner_train.ipynb',
 'urm_ner.ipynb',
 'urm_ner_visual.ipynb',
 'evaluate_urm.ipynb']

In [4]:
from polimi.utils._custom import load_sparse_csr

ner_path = dpath.parent.joinpath('urm').joinpath('ner').joinpath('small')
algo_path = ner_path.joinpath('algo').joinpath('train')

URM_train = load_sparse_csr(ner_path.joinpath('URM_train.npz'))
URM_train_val = load_sparse_csr(ner_path.joinpath('URM_train_val.npz'))
URM_val =  load_sparse_csr(ner_path.joinpath('URM_validation.npz'))
URM_test = load_sparse_csr(ner_path.joinpath('URM_test.npz'))

File loaded at: /mnt/ebs_volume/recsys2024/urm/ner/small/URM_train.npz
File loaded at: /mnt/ebs_volume/recsys2024/urm/ner/small/URM_train_val.npz
File loaded at: /mnt/ebs_volume/recsys2024/urm/ner/small/URM_validation.npz
File loaded at: /mnt/ebs_volume/recsys2024/urm/ner/small/URM_test.npz


In [6]:
URM_train

<15143x43774 sparse matrix of type '<class 'numpy.float64'>'
	with 13187741 stored elements in Compressed Sparse Row format>

In [7]:
URM_val

<15143x43774 sparse matrix of type '<class 'numpy.float64'>'
	with 1950637 stored elements in Compressed Sparse Row format>

In [21]:
# from polimi.utils._custom import load_best_optuna_params

# rp3params = load_best_optuna_params('RP3betaRecommender-ner-small-ndcg100')
# userknnparams = load_best_optuna_params('UserKNNCFRecommender-ner-small-ndcg100')
# itemknnparams = load_best_optuna_params('ItemKNNCFRecommender-ner-small-ndcg100')
# puresvditemparams = load_best_optuna_params('PureSVDItemRecommender-ner-small-ndcg100')
# rp3params, userknnparams, itemknnparams, puresvditemparams

In [22]:

# train_recommender(URM_train, RP3betaRecommender, rp3params, file_name='RP3betaRecommender-ner-small-ndcg100', output_dir=algo_path)
# train_recommender(URM_train, UserKNNCFRecommender, userknnparams, file_name='UserKNNCFRecommender-ner-small-ndcg100', output_dir=algo_path)
# train_recommender(URM_train, ItemKNNCFRecommender, itemknnparams, file_name='ItemKNNCFRecommender-ner-small-ndcg100', output_dir=algo_path)
# train_recommender(URM_train, PureSVDItemRecommender, puresvditemparams, file_name='PureSVDItemRecommender-ner-small-ndcg100', output_dir=algo_path)

# Build rec sys features

In [7]:
pure_svd = PureSVDRecommender(URM_train)
pure_svd.fit(num_factors=20)

PureSVDRecommender: URM Detected 1 ( 0.0%) users with no interactions.
PureSVDRecommender: URM Detected 18909 (43.2%) items with no interactions.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 12.78 sec


In [11]:
from polimi.utils._urm import build_ner_mapping, build_user_id_mapping, build_articles_with_processed_ner, _build_batch_ner_interactions
user_id_mapping = build_user_id_mapping(history_train)
ap = build_articles_with_processed_ner(articles)
ner_mapping = build_ner_mapping(ap)
ap = ap.with_columns(
    pl.col('ner_clusters').list.eval(pl.element().replace(ner_mapping['ner'], ner_mapping['ner_index'], default=None)).list.drop_nulls().alias('ner_clusters_index'),
)

In [12]:
train_ds = behaviors_train
train_ds = train_ds\
    .rename({'article_ids_inview': 'candidate_ids'})\
    .with_columns(
        pl.col('candidate_ids').list.eval(pl.element().replace(ap['article_id'], ap['ner_clusters_index'], default=[])).alias('candidate_ner_index'),
        pl.col('user_id').replace(user_id_mapping['user_id'], user_id_mapping['user_index'], default=None).alias('user_index')
    ).select('impression_id', 'user_id', 'user_index', 'candidate_ids', 'candidate_ner_index')
train_ds.head(2)

impression_id,user_id,user_index,candidate_ids,candidate_ner_index
u32,u32,u32,list[i32],list[list[u32]]
149474,139836,747,"[9778623, 9778682, … 9778728]","[[1016, 3366, … 41304], [3827, 4679, … 39272], … [14122, 14127, … 42433]]"
150528,143471,773,"[9778718, 9778728, … 9778682]","[[5927, 23803, … 42713], [14122, 14127, … 42433], … [3827, 4679, … 39272]]"


In [14]:
all_users = user_id_mapping['user_index'].unique().sort().to_list()
all_items = ner_mapping['ner_index'].unique().sort().to_list()
len(all_users), len(all_items)

(15143, 43774)

In [16]:
from tqdm import tqdm
train_ds = pl.concat([
    slice.explode(['candidate_ids', 'candidate_ner_index'])\
        .filter(pl.col('candidate_ner_index').list.len() > 0)\
        .group_by(['impression_id', 'user_id', 'user_index']).agg(pl.all())
    for slice in tqdm(train_ds.iter_slices(100000), total=train_ds.shape[0]//100000)
])
train_ds.head(2)

3it [00:04,  1.37s/it]                       


impression_id,user_id,user_index,candidate_ids,candidate_ner_index
u32,u32,u32,list[i32],list[list[u32]]
158024899,1699113,9885,"[9778007, 9778110, … 9775894]","[[8308, 10671, … 40982], [11010, 11146, … 42069], … [5107, 5110, … 42172]]"
94964662,2363201,13847,"[9521552, 9778500, … 9778448]","[[19970, 26603, … 37870], [1504, 3664, … 36851], … [607, 9316, … 41306]]"


In [17]:
train_ds = train_ds.sort('user_id')
train_ds.head(2)

impression_id,user_id,user_index,candidate_ids,candidate_ner_index
u32,u32,u32,list[i32],list[list[u32]]
385198819,10068,0,"[9777769, 9762352, … 9779737]","[[3666, 9051, … 40567], [927, 8308, … 38774], … [11460, 14656, … 43299]]"
556008624,10068,0,"[9779813, 9779860, … 9779891]","[[9151, 10671, … 38682], [1567, 7153, … 42371], … [6518, 7705, … 34548]]"


In [20]:
pl.concat([
    slice.explode(['candidate_ids', 'candidate_ner_index'])\
    .with
    .explode('candidate_ner_index')
    for slice in tqdm(train_ds[:20000].iter_slices(10000), total=train_ds.shape[0] // 10000)
])

  9%|▊         | 2/23 [00:00<00:02,  8.59it/s]


impression_id,user_id,user_index,candidate_ids,candidate_ner_index
u32,u32,u32,i32,u32
385198819,10068,0,9777769,3666
385198819,10068,0,9777769,9051
385198819,10068,0,9777769,9314
385198819,10068,0,9777769,9911
385198819,10068,0,9777769,9932
…,…,…,…,…
377566096,236781,1336,9771168,23753
377566096,236781,1336,9771168,26779
377566096,236781,1336,9771168,33196


In [35]:
import numpy as np
x = train_ds['user_index'].unique().to_list()
user_index_id_map = dict(zip(np.arange(0, len(x)), x))
user_index_id_map

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 19: 19,
 20: 20,
 21: 21,
 22: 22,
 23: 23,
 24: 24,
 25: 25,
 26: 26,
 27: 27,
 28: 28,
 29: 29,
 30: 30,
 31: 31,
 32: 32,
 33: 33,
 34: 34,
 35: 35,
 36: 36,
 37: 37,
 38: 38,
 39: 39,
 40: 40,
 41: 41,
 42: 42,
 43: 43,
 44: 44,
 45: 45,
 46: 46,
 47: 47,
 48: 48,
 49: 49,
 50: 50,
 51: 51,
 52: 52,
 53: 53,
 54: 54,
 55: 55,
 56: 56,
 57: 57,
 58: 58,
 59: 59,
 60: 60,
 61: 61,
 62: 62,
 63: 63,
 64: 64,
 65: 65,
 66: 66,
 67: 67,
 68: 68,
 69: 69,
 70: 70,
 71: 71,
 72: 72,
 73: 73,
 74: 74,
 75: 75,
 76: 76,
 77: 77,
 78: 78,
 79: 79,
 80: 80,
 81: 81,
 82: 82,
 83: 83,
 84: 84,
 85: 85,
 86: 86,
 87: 87,
 88: 88,
 89: 89,
 90: 90,
 91: 91,
 92: 92,
 93: 93,
 94: 94,
 95: 95,
 96: 96,
 97: 97,
 98: 98,
 99: 99,
 100: 100,
 101: 101,
 102: 102,
 103: 103,
 104: 104,
 105: 105,
 106: 106,
 107: 107,
 108: 108,
 109: 109,
 110: 110,

In [42]:
pure_svd._compute_item_score([0, 1], [1, 2])[:, [1, 2]]

array([[-5.8590324e-08,  1.6068752e-03],
       [-3.4997387e-07,  2.1925143e-03]], dtype=float32)

In [45]:
recs = [pure_svd]
for slice in tqdm(train_ds[:20000].iter_slices(10000), total=train_ds.shape[0] // 10000):
    user_indexes = train_ds['user_index'].unique().to_list()
    user_index_id_map = dict(zip(user_indexes,np.arange(0, len(user_indexes))))
    scores = {rec.RECOMMENDER_NAME: rec._compute_item_score(user_indexes, all_items)[:, all_items] for rec in recs}
    break
scores

  0%|          | 0/23 [00:14<?, ?it/s]


{'PureSVDRecommender': array([[ 2.5492281e-02, -5.8590324e-08,  1.6068752e-03, ...,
          3.2954734e-02,  9.6922554e-04,  0.0000000e+00],
        [-6.9945768e-02, -3.4997387e-07,  2.1925143e-03, ...,
          6.2792525e-02, -1.9334888e-03,  0.0000000e+00],
        [ 1.2526023e-01,  1.8451232e-07, -4.2353696e-04, ...,
          2.4620036e-02,  2.0885251e-03,  0.0000000e+00],
        ...,
        [ 3.3386130e-02,  2.9893627e-07, -1.4291659e-03, ...,
         -2.1126937e-02,  1.6923860e-03,  0.0000000e+00],
        [-1.5101503e-04, -3.5595605e-08,  1.3351100e-04, ...,
          2.6617931e-02,  5.7854044e-04,  0.0000000e+00],
        [ 1.6303912e-02,  2.6173910e-07, -1.4627439e-03, ...,
          2.4987306e-02,  1.9803855e-03,  0.0000000e+00]], dtype=float32)}

In [29]:
from tqdm import tqdm
recs = [rp3beta]

impression_scores = pl.concat([
    slice.with_columns(
            *[pl.col('candidate_ner_index').list.eval(
                pl.element().list.eval(pl.element().replace(all_items, rec._compute_item_score(user_index, all_items)[0, all_items], default=None))
            ).alias(f"{rec.RECOMMENDER_NAME}_scores") for rec in recs]
        ).drop('user_index', 'candidate_ner_index')
for user_index, slice in tqdm(train_ds[:1000].partition_by(['user_index'], as_dict=True).items(), total=train_ds['user_index'].n_unique())
])
impression_scores.head(3)

  6%|▌         | 855/15143 [00:08<02:24, 98.62it/s] 


impression_id,user_id,candidate_ids,RP3betaRecommender_scores
u32,u32,list[i32],list[list[f32]]
3087492,2254761,"[9771242, 9769497, … 9771253]","[[0.0, 0.76192, … 0.374787], [0.143495, 0.412684, … 0.132622], … [0.0, 0.0, … 0.007103]]"
7891052,2254761,"[9777693, 9364014, … 9777319]","[[0.399421, 0.619761, … 0.0], [0.0], … [0.050296, 0.737277, … 0.166674]]"
10739509,1352157,"[9465810, 9527358, … 9755800]","[[0.0, 0.156983, … 0.286112], [0.0, 0.172582, … 0.0], … [0.878252, 0.005664, … 0.0]]"


In [1]:
behaviors_train

NameError: name 'behaviors_train' is not defined

In [30]:
from polimi.utils._polars import reduce_polars_df_memory_size
scores_cols = [col for col in impression_scores.columns if col.endswith('_scores')]
df = impression_scores.with_columns(
        *[pl.col(col).list.eval(pl.element().list.sum()).alias(f'sum_{col}') for col in scores_cols],
        *[pl.col(col).list.eval(pl.element().list.max()).alias(f'max_{col}') for col in scores_cols],
        *[pl.col(col).list.eval(pl.element().list.mean()).alias(f'mean_{col}') for col in scores_cols],
).with_columns(
    pl.all().exclude(['impression_id', 'user_id', 'candidate_ids'] + scores_cols).list.eval(pl.element().truediv(pl.element().max()).fill_nan(0.0)), #inf norm
).drop(scores_cols)

df = reduce_polars_df_memory_size(df)
df = df.sort(['impression_id', 'user_id'])\
    .explode(pl.all().exclude(['impression_id', 'user_id']))\
    .rename({'candidate_ids': 'article'})

df.head(3)

Memory usage of dataframe is 0.20 MB
Memory usage after optimization is: 0.20 MB
Decreased by 0.0%


impression_id,user_id,article,sum_RP3betaRecommender_scores,max_RP3betaRecommender_scores,mean_RP3betaRecommender_scores
u32,u32,i32,f32,f32,f32
153068,151570,9778657,1.0,0.934113,1.0
153068,151570,9778669,0.022829,0.1231,0.041853
153068,151570,9772866,0.164515,0.399589,0.139205


In [ ]:
df[0]['sum_PureSVDRecommender_scores'].list.min()

# Test API

In [11]:
from polimi.scripts.build_urm_ner_scores_features import main

main(dpath, ner_path, algo_path, Path('.'))

File loaded at: /mnt/ebs_volume/recsys2024/urm/ner/small/URM_train.npz
RP3betaRecommender: URM Detected 1 ( 0.0%) users with no interactions.
RP3betaRecommender: URM Detected 18909 (43.2%) items with no interactions.
RP3betaRecommender: Similarity column 43774 (100.0%), 363.84 column/sec. Elapsed time 2.01 min
RP3betaRecommender: Saving model in file '/mnt/ebs_volume/recsys2024/urm/ner/small/algo/train/trainRP3betaRecommender-ner-small-ndcg100_new'
RP3betaRecommender: Saving complete
PureSVDItemRecommender: URM Detected 1 ( 0.0%) users with no interactions.
PureSVDItemRecommender: URM Detected 18909 (43.2%) items with no interactions.
PureSVDItemRecommender: Computing SVD decomposition...


KeyboardInterrupt: 

# Evaluate Models

In [ ]:
rp3beta = RP3betaRecommender(URM_train)
user_knn = UserKNNCFRecommender(URM_train)
# item_knn = ItemKNNCFRecommender(URM_train)
# pure_svd_item = PureSVDItemRecommender(URM_train)
rp3beta.load_model(str(algo_path), file_name='RP3betaRecommender-ner-small-ndcg100')
user_knn.load_model(str(algo_path), file_name='UserKNNCFRecommender-ner-small-ndcg100')
# item_knn.load_model(str(algo_path), file_name='ItemKNNCFRecommender-ner-small-ndcg100')
# pure_svd_item.load_model(str(algo_path), file_name='PureSVDItemRecommender-ner-small-ndcg100')

In [ ]:
from RecSys_Course_AT_PoliMi.Evaluation.Evaluator import EvaluatorHoldout

cutoff = 100
metric = 'NDCG'
evaluator = EvaluatorHoldout(URM_val, cutoff_list=[cutoff, 10], exclude_seen=False)
test_evaluator = EvaluatorHoldout(URM_test, cutoff_list=[cutoff, 10], exclude_seen=False)

In [ ]:
# user_knn = UserKNNCFRecommender(URM_train)
# user_knn.fit()
result_df, _ = evaluator.evaluateRecommender(user_knn)
result_df.loc[cutoff][metric.upper()]

In [ ]:
result_df

In [ ]:
user_knn = UserKNNCFRecommender(URM_train_val)
user_knn.fit(**userknnparams)
result_df, _ = test_evaluator.evaluateRecommender(user_knn)
result_df.loc[cutoff][metric.upper()]